In [24]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("spark://192.168.2.97:7077") \
        .appName("Group-6-Question-1-Claude")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.cores.max", 2)\
        .getOrCreate()

In [22]:
from pyspark.sql.functions import trim, regexp_replace, lower, explode, trim, split
from pyspark.sql.types import StructType, StructField, StringType

In [23]:
df = spark.read.json("../RC_2006-01.json")
df.show(10)

+--------------------+----------+
|                body| subreddit|
+--------------------+----------+
|early 2006 a prob...|reddit.com|
|If you are going ...|reddit.com|
|Microsoft hates i...|reddit.com|
|this looks intere...|reddit.com|
|I have nothing bu...|reddit.com|
|And by "Microsoft...|reddit.com|
|This is exactly t...|reddit.com|
|           [deleted]|reddit.com|
|Because this one ...|reddit.com|
|                  ok|reddit.com|
+--------------------+----------+
only showing top 10 rows



In [62]:
my_df_lower = df.select(lower("body").alias("lowercase_body"))

charachters_to_delete = "[\.,\[\]\(\):_\-!?\'\+=;/&{}@$#*\"\\\\%><|~¨´¤]"

my_df_clean = my_df_lower.withColumn("lowercase_body", regexp_replace(my_df_lower.lowercase_body, charachters_to_delete, ""))

my_df_words = my_df_clean.select(explode(split("lowercase_body", "\s+")).alias("word"))

word_counts = my_df_words.groupBy("word").count().orderBy("count", ascending=False)

my_df_clean.show(10)

+--------------------+
|      lowercase_body|
+--------------------+
|early 2006 a prob...|
|if you are going ...|
|microsoft hates i...|
|this looks intere...|
|i have nothing bu...|
|and by microsoft ...|
|this is exactly t...|
|             deleted|
|because this one ...|
|                  ok|
+--------------------+
only showing top 10 rows



In [63]:
word_counts.show(1000)

+--------------+-----+
|          word|count|
+--------------+-----+
|           the| 6443|
|            to| 3875|
|             a| 3445|
|            of| 3201|
|           and| 2768|
|            is| 2737|
|          that| 2429|
|             i| 2332|
|            in| 2016|
|            it| 1909|
|           you| 1444|
|           for| 1318|
|          this| 1250|
|           not| 1025|
|           are|  999|
|            be|  980|
|           but|  947|
|            on|  935|
|          have|  875|
|          with|  830|
|            as|  830|
|            if|  695|
|          they|  671|
|            or|  658|
|           its|  654|
|         about|  604|
|           was|  589|
|        people|  571|
|          what|  554|
|          like|  549|
|          just|  545|
|            so|  541|
|            at|  539|
|            an|  525|
|          more|  477|
|       deleted|  475|
|           all|  475|
|          from|  474|
|           can|  470|
|         would|  461|
|          

In [ ]:
spark_session.stop()